In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('blackfridaysales').getOrCreate()

In [ ]:
salestrain=spark.read.csv('/content/train.csv',inferSchema=True,header=True)

In [ ]:
salestest=spark.read.csv('/content/test.csv',inferSchema=True,header=True)

In [ ]:
salestrain.show(5)

+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender| Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+----+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|1000001| P00069042|     F|0-17|        10|            A|                         2|             0|                 3|              null|              null|    8370|
|1000001| P00248942|     F|0-17|        10|            A|                         2|             0|                 1|                 6|                14|   15200|
|1000001| P00087842|     F|0-17|        10|            A|                         2|             0|                12|              null|              null|    1422|
|100

In [ ]:
salestrain.describe().show()

+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|           User_ID|Product_ID|Gender|   Age|       Occupation|City_Category|Stay_In_Current_City_Years|    Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|         Purchase|
+-------+------------------+----------+------+------+-----------------+-------------+--------------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|            474330|    474330|474330|474330|           474330|       474330|                    474330|            474330|            474330|            327086|            145056|           474330|
|   mean|1002992.2208378133|      null|  null|  null| 8.07891763118504|         null|        1.4685511539305447|0.4094596588872726| 5.297149663736217|  9.84450572632274|  1

In [ ]:
from pyspark.sql.functions import isnull,when,count,col

In [ ]:
salestrain.select([count(when(isnull(c),c)).alias(c) for c in salestrain.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|Purchase|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|            173638|            383247|       0|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+--------+



In [ ]:
salestrain.groupBy('Product_Category_2').count().show()
# Since there are many missing values, Impute with 99 as NA code

+------------------+------+
|Product_Category_2| count|
+------------------+------+
|                12|  5528|
|                13| 10531|
|                 6| 16466|
|                16| 43255|
|                 3|  2884|
|                 5| 26235|
|                15| 37855|
|                 9|  5693|
|                17| 13320|
|                 4| 25677|
|                 8| 64088|
|                 7|   626|
|                10|  3043|
|                11| 14134|
|                14| 55108|
|                 2| 49217|
|                18|  2770|
|              null|173638|
+------------------+------+



In [ ]:
salestrain=salestrain.na.fill(value=99,subset='Product_Category_2')

In [ ]:
salestrain.groupBy('Product_Category_3').count().show()

+------------------+------+
|Product_Category_3| count|
+------------------+------+
|                12|  9246|
|                13|  5459|
|                16| 32636|
|                 6|  4890|
|                 3|   613|
|                 5| 16658|
|                15| 28013|
|                17| 16702|
|                 9| 11579|
|                 4|  1875|
|                 8| 12562|
|                10|  1726|
|                11|  1805|
|                14| 18428|
|                18|  4629|
|              null|383247|
+------------------+------+



In [ ]:
salestrain=salestrain.na.fill(value=99,subset='Product_Category_3')
# Since there are many missing values, Impute with 99 as NA code

In [ ]:
salestrain.groupBy('Product_Category_3').count().show()

+------------------+------+
|Product_Category_3| count|
+------------------+------+
|                12|  9246|
|                13|  5459|
|                16| 32636|
|                 6|  4890|
|                 3|   613|
|                 5| 16658|
|                15| 28013|
|                17| 16702|
|                 9| 11579|
|                 4|  1875|
|                 8| 12562|
|                10|  1726|
|                11|  1805|
|                14| 18428|
|                99|383247|
|                18|  4629|
+------------------+------+



In [ ]:
salestest.select([count(when(isnull(c),c)).alias(c) for c in salestest.columns]).show()

+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|User_ID|Product_ID|Gender|Age|Occupation|City_Category|Stay_In_Current_City_Years|Marital_Status|Product_Category_1|Product_Category_2|Product_Category_3|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+
|      0|         0|     0|  0|         0|            0|                         0|             0|                 0|             72344|            162562|
+-------+----------+------+---+----------+-------------+--------------------------+--------------+------------------+------------------+------------------+



In [ ]:
salestest=salestest.na.fill(value=99,subset='Product_Category_2')

In [ ]:
salestest=salestest.na.fill(value=99,subset='Product_Category_3')

In [ ]:
salestrain.printSchema()     # Variable type

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [ ]:
indexer=[StringIndexer(inputCol=column,outputCol=column+'index').fit(salestrain)
for column in list(set(salestrain.columns)-set(['Purchase','User_ID','Product_ID']))]

In [ ]:
pipeline=Pipeline(stages=indexer)

In [ ]:
salestraindf=pipeline.fit(salestrain).transform(salestrain)

In [ ]:
salestestdf=pipeline.fit(salestest).transform(salestest)

In [ ]:
salestraindf.columns

['User_ID',
 'Product_ID',
 'Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status',
 'Product_Category_1',
 'Product_Category_2',
 'Product_Category_3',
 'Purchase',
 'Genderindex',
 'Product_Category_3index',
 'Marital_Statusindex',
 'Product_Category_2index',
 'Occupationindex',
 'Product_Category_1index',
 'City_Categoryindex',
 'Stay_In_Current_City_Yearsindex',
 'Ageindex']

In [ ]:
columnstodrop=['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years',
               'Marital_Status','Product_Category_1','Product_Category_2','Product_Category_3']

In [ ]:
salestraindf=salestraindf.drop(*columnstodrop)

In [ ]:
from pyspark.ml.feature import RFormula

In [ ]:
formula=RFormula(formula="Purchase~.",featuresCol='features',labelCol='label')

In [ ]:
salestraindf=formula.fit(salestraindf).transform(salestraindf)

In [ ]:
salestraindf.select('features','label').show(5)

+--------------------+-------+
|            features|  label|
+--------------------+-------+
|[1.0,0.0,0.0,0.0,...| 8370.0|
|[1.0,3.0,0.0,8.0,...|15200.0|
|[1.0,0.0,0.0,0.0,...| 1422.0|
|[1.0,0.0,0.0,2.0,...| 1057.0|
|[0.0,0.0,0.0,0.0,...| 7969.0|
+--------------------+-------+
only showing top 5 rows



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
reg=LinearRegression(featuresCol='features',labelCol='label')

In [ ]:
regmodel=reg.fit(salestraindf)

In [ ]:
regmodel.summary.r2adj

0.0635630011793038

In [ ]:
regmodel.summary.rootMeanSquaredError

4860.7598698134

In [ ]:
salestestdf=salestestdf.drop(*columnstodrop)

In [ ]:
salestestdf.columns

['Genderindex',
 'Product_Category_3index',
 'Marital_Statusindex',
 'Product_Category_2index',
 'Occupationindex',
 'Product_Category_1index',
 'City_Categoryindex',
 'Stay_In_Current_City_Yearsindex',
 'Ageindex']

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler=VectorAssembler(inputCols=['Genderindex',
 'Product_Category_3index',
 'Marital_Statusindex',
 'Product_Category_2index',
 'Occupationindex',
 'Product_Category_1index',
 'City_Categoryindex',
 'Stay_In_Current_City_Yearsindex',
 'Ageindex'],outputCol='features')

In [ ]:
salestestdf=assembler.transform(salestestdf)

In [ ]:
regpredict=regmodel.transform(salestestdf)

In [ ]:
regpredict.select('prediction').toPandas().to_csv("reg.csv")

In [ ]:
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
tree=DecisionTreeRegressor(featuresCol='features',labelCol='label')

In [ ]:
treemodel=tree.fit(salestraindf)

In [ ]:
treemodel=treemodel.transform(salestraindf)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='label',metricName='rmse')

In [ ]:
evaluator.evaluate(treemodel)

2997.772179128243

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
RF=RandomForestRegressor(featuresCol='features',labelCol='label')

Py4JError: ignored

In [ ]:
RFmodel=RF.fit(salestraindf)

KeyboardInterrupt: ignored

In [ ]:
RFmodel=RFmodel.transform(salestraindf)

In [ ]:
evaluator.evaluate(RFmodel)

3191.1957442400003

In [ ]:
from pyspark.ml.regression import GBTRegressor

In [ ]:
GB=GBTRegressor(featuresCol='features',labelCol='label')

In [ ]:
GBmodel=GB.fit(salestraindf)

In [ ]:
GBmodel=GBmodel.transform(salestraindf)

In [ ]:
evaluator.evaluate(GBmodel)

2928.9377873687395